In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
os.environ["KERAS_BACKEND"] = "tensorflow"

import uproot
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import networkx as nx
import copy
import math
import keras
import glob
import sys
import datetime
from collections import Counter

import numba

In [ ]:
from itertools import combinations

In [ ]:
%matplotlib inline

In [ ]:
@numba.njit
def get_index_triu_vector(i, j, vecsize):
    k = j - i - 1
    k += i*vecsize
    missing = int(i*(i+1)/2)
    k -= missing
    return k

In [ ]:
#This file created using test/ntuplizer.py
fn = "../data/TTbar_run3/step3_ntuple_1.root"
fi = uproot.open(fn)
tree = fi.get("pftree")
data = tree.arrays(tree.keys())
data = {str(k, 'ascii'): v for k, v in data.items()}

## Full event

In [ ]:
iev=0

#remove tracks for which the 
trks_outer_valid = (data["tracks_outer_eta"][iev]!= 0) & (data["tracks_outer_phi"][iev]!= 0)
trks_inner_valid = (data["tracks_inner_eta"][iev]!= 0) & (data["tracks_inner_phi"][iev]!= 0)

plt.figure(figsize=(15, 15))
plt.scatter(data["clusters_eta"][iev], data["clusters_phi"][iev], marker="s", label="cluster (N={0})".format(len(data["clusters_eta"][iev])))
plt.scatter(data["tracks_eta"][iev], data["tracks_phi"][iev], marker="x", label="general track (N={0})".format(len(data["tracks_eta"][iev])))
plt.scatter(data["tracks_inner_eta"][iev][trks_inner_valid], data["tracks_inner_phi"][iev][trks_inner_valid], marker="x", label="inner track (N={0})".format(np.sum(trks_inner_valid)))
plt.scatter(data["tracks_outer_eta"][iev][trks_outer_valid], data["tracks_outer_phi"][iev][trks_outer_valid], marker="x", label="outer track (N={0})".format(np.sum(trks_outer_valid)))
plt.scatter(data["pfcands_eta"][iev], data["pfcands_phi"][iev], marker="o", alpha=0.2, s=100, color="black", label="candidate (N={0})".format(len(data["pfcands_eta"][iev])))
plt.scatter(data["genparticles_eta"][iev], data["genparticles_phi"][iev], marker="o", alpha=0.2, s=200, color="purple", label="genparticle (N={0})".format(len(data["genparticles_eta"][iev])))

plt.xlabel("$\eta$")
plt.ylabel("$\phi$")
plt.legend()

In [ ]:
plt.figure(figsize=(5, 5))

plt.title("ECAL surface")
trks_inner_valid = (data["tracks_inner_eta"][iev]!= 0) & (data["tracks_inner_phi"][iev]!= 0)
clusters_ecal = data["clusters_type"][iev] == 4

plt.scatter(data["tracks_inner_eta"][iev][trks_inner_valid], data["tracks_inner_phi"][iev][trks_inner_valid], marker="x", label="inner track (N={0})".format(np.sum(trks_inner_valid)))
plt.scatter(data["clusters_eta"][iev][clusters_ecal], data["clusters_phi"][iev][clusters_ecal], marker="s", label="cluster (N={0})".format(sum(clusters_ecal)), alpha=0.5)

In [ ]:
plt.figure(figsize=(5, 5))

plt.title("HCAL surface")
trks_outer_valid = (data["tracks_outer_eta"][iev]!= 0) & (data["tracks_outer_phi"][iev]!= 0)
clusters_ecal = data["clusters_type"][iev] == 5

plt.scatter(data["tracks_outer_eta"][iev][trks_outer_valid], data["tracks_outer_phi"][iev][trks_outer_valid], marker="x", label="inner track (N={0})".format(np.sum(trks_inner_valid)))
plt.scatter(data["clusters_eta"][iev][clusters_ecal], data["clusters_phi"][iev][clusters_ecal], marker="s", label="cluster (N={0})".format(sum(clusters_ecal)), alpha=0.5)

## Detector subsystems

In [ ]:
#Event number to plot
i = 0

#minimum pt of the pf candidate
pfcand_ptcut = 2


c_eta = data["clusters_eta"]
c_phi = data["clusters_phi"]
c_en = data["clusters_energy"]
c_type = data["clusters_type"]

t_o_eta = data["tracks_outer_eta"]
t_o_phi = data["tracks_outer_phi"]
t_i_eta = data["tracks_inner_eta"]
t_i_phi = data["tracks_inner_phi"]
t_q = np.sign(data["tracks_qoverp"])
t_en = 1.0/np.abs(data["tracks_qoverp"])

cand_eta = data["pfcands_eta"]
cand_phi = data["pfcands_phi"]
cand_en = data["pfcands_pt"]
cand_charge = data["pfcands_charge"]

g_eta = data["genparticles_eta"]
g_phi = data["genparticles_phi"]
g_en = data["genparticles_pt"]

plt.figure(figsize=(15,10))

ax = plt.subplot(2,3,1)
for ct in [2,3,4,5,8,9,11]:
    cluster_type_sel = c_type[i]==ct
    plt.scatter(
        c_eta[i][cluster_type_sel],
        c_phi[i][cluster_type_sel],
        s=c_en[i][cluster_type_sel],
        marker="s", alpha=0.5, label="{0}".format(ct))
plt.xlabel("$\eta$")
plt.ylabel("$\phi$")
plt.title("calo clusters (N={0})".format(len(c_eta[i])))
plt.xlim(-5,5)
plt.ylim(-5,5)
#plt.legend(loc="best", ncol=2)

ax = plt.subplot(2,3,2)
for track_charge in [-1, 1]:
    track_charge_sel = t_q[i] == track_charge
    plt.scatter(t_o_eta[i][track_charge_sel], t_o_phi[i][track_charge_sel], s=t_en[i][track_charge_sel], marker="o", alpha=0.5)
plt.xlabel("outer $\eta$")
plt.ylabel("outer $\phi$")
plt.title("tracks outer hit (N={0})".format(len(t_o_eta[i])))
plt.xlim(-5,5)
plt.ylim(-5,5)

ax = plt.subplot(2,3,3)
for track_charge in [-1, 1]:
    track_charge_sel = t_q[i] == track_charge
    plt.scatter(t_i_eta[i][track_charge_sel], t_i_phi[i][track_charge_sel], s=t_en[i][track_charge_sel], marker="o", alpha=0.5)
plt.xlabel("inner $\eta$")
plt.ylabel("inner $\phi$")
plt.title("tracks inner hit (N={0})".format(len(t_o_eta[i])))
plt.xlim(-5,5)
plt.ylim(-5,5)

ax = plt.subplot(2,3,4)
for _cand_charge in [-1, 1, 0]:
    cand_charge_sel = cand_charge[i] == _cand_charge
    plt.scatter(cand_eta[i][cand_charge_sel], cand_phi[i][cand_charge_sel], s=cand_en[i][cand_charge_sel], marker="o", alpha=0.5)
plt.xlabel("$\eta$")
plt.ylabel("$\phi$")
plt.title("PF candidates (N={0})".format(len(cand_eta[i])))
plt.xlim(-5,5)
plt.ylim(-5,5)

ax = plt.subplot(2,3,5)
cand_pt_sel = (cand_en[i] > pfcand_ptcut)
for _cand_charge in [-1, 1, 0]:
    cand_charge_sel = (cand_charge[i] == _cand_charge) & (cand_pt_sel)
    plt.scatter(
        cand_eta[i][cand_charge_sel],
        cand_phi[i][cand_charge_sel],
        s=cand_en[i][cand_charge_sel], marker="o", alpha=0.5)
plt.xlabel("$\eta$")
plt.ylabel("$\phi$")
plt.title("PF candidates with $p_T > {0}$ GeV, (N={1})".format(pfcand_ptcut, len(cand_eta[i][cand_pt_sel])))
plt.xlim(-5,5)
plt.ylim(-5,5)

ax = plt.subplot(2,3,6)
plt.scatter(g_eta[i], g_phi[i], s=g_en[i], marker="x", alpha=0.5)
plt.xlabel("$\eta$")
plt.ylabel("$\phi$")
plt.title("genparticles (N={0})".format(len(g_eta[i])))

plt.xlim(-5,5)
plt.ylim(-5,5)
plt.tight_layout()

In [ ]:
from collections import Counter

In [ ]:
all_blocks_counts_0 = []
all_blocks_counts_1 = []
all_blocks_counts_2 = []
all_blocks_counts_3 = []

for iev in range(len(data["clusters_iblock"])):
    bls = list(data["clusters_iblock"][iev])
    bls2 = list(data["tracks_iblock"][iev])
    bls_counts = sorted([v for k, v in Counter(bls+bls2).items()], reverse=True)
    all_blocks_counts_0 += [bls_counts[0]]
    all_blocks_counts_1 += [bls_counts[1]]
    all_blocks_counts_2 += [bls_counts[2]]
    all_blocks_counts_3 += [bls_counts[3]]

In [ ]:
len(np.unique(list(data["clusters_iblock"][iev]) + list(data["tracks_iblock"][iev])))

In [ ]:
bins = np.linspace(0,1.5*max(all_blocks_counts_0), 20)
plt.hist(all_blocks_counts_0, bins=bins, histtype="step", lw=2, label="first")
plt.hist(all_blocks_counts_1, bins=bins, histtype="step", lw=2, label="second")
plt.hist(all_blocks_counts_2, bins=bins, histtype="step", lw=2, label="third")

plt.yscale("log")
plt.legend(loc="best")
plt.xlabel("elements per block")